In [ ]:
import sklearn

from keras.layers import LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
import numpy as np
import pickle

In [ ]:
with open('org_glv_w2v.pkl', 'rb') as file:
  glv_vec = pickle.load(file)
  w2v_vec = pickle.load(file)
  y = pickle.load(file)
  random_state = pickle.load(file)
  test_size = pickle.load(file)
file.close()

In [ ]:
with open('org_gen_aug_w2v.pkl', 'rb') as file:
  new_x_train_w2v = pickle.load(file)
  new_y_train_w2v = pickle.load(file)
  aug_vec_w2v = pickle.load(file)
  aug_y_w2v = pickle.load(file)
file.close()

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [ ]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(w2v_vec, y, test_size=0.25, random_state=41)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Bidirectional, GRU, concatenate, Conv1D
from keras import layers
from keras.models import Model

In [ ]:
def rnn(RNN, input_shape):
  if RNN == 'LSTM': RNN = LSTM
  elif RNN == 'GRU': RNN = GRU
  
  """Define the model."""
  model = Sequential()
  model.add(Bidirectional(RNN(input_shape[-1], dropout=0.4, recurrent_dropout=0.4, input_shape=[1, input_shape[-1]], return_sequences=True)))
  model.add(Bidirectional(RNN(64, recurrent_dropout=0.4)))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='relu'))

  model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
  model.build(input_shape)
  model.summary()

  return model

In [ ]:
rnn1 = rnn('LSTM', X_train_w2v.shape)
rnn1.fit(X_train_w2v, y_train_w2v, batch_size=64, epochs=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (8037, 1, 600)           1442400   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (8037, 128)              340480    
 nal)                                                            
                                                                 
 dropout (Dropout)           (8037, 128)               0         
                                                                 
 dense (Dense)               (8037, 1)                 129       
                                                                 
Total params: 1,783,009
Trainable params: 1,783,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
126/126 [=========================

In [ ]:
rnn2 = rnn('GRU', X_train_w2v.shape)
rnn2.fit(X_train_w2v, y_train_w2v, batch_size=64, epochs=100)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (8037, 1, 600)           1083600   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (8037, 128)              255744    
 nal)                                                            
                                                                 
 dropout (Dropout)           (8037, 128)               0         
                                                                 
 dense (Dense)               (8037, 1)                 129       
                                                                 
Total params: 1,339,473
Trainable params: 1,339,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
126/126 [=========================

In [ ]:
def test_rnn_model(model, x_test, y_test):
  y_pred = model.predict(x_test)
  y_pred = np.around(y_pred)

  result_aug = cohen_kappa_score(y_test,y_pred,weights='quadratic')
  print("Kappa Score: {}".format(result_aug))

In [ ]:
test_rnn_model(rnn1, X_test_w2v, y_test_w2v)

84/84 [==============================] - 2s 11ms/step
Kappa Score: 0.6819689213038195


In [ ]:
test_rnn_model(rnn2, X_test_w2v, y_test_w2v)


84/84 [==============================] - 2s 10ms/step
Kappa Score: 0.6756829139599481


In [ ]:
_, new_x_test_w2v, _, new_y_test_w2v = train_test_split(new_x_train_w2v, new_y_train_w2v, test_size=0.25, random_state=41)


In [ ]:
test_rnn_model(rnn1, new_x_test_w2v, new_y_test_w2v)


65/65 [==============================] - 1s 11ms/step
Kappa Score: 0.11938740760811051


In [ ]:
test_rnn_model(rnn2, new_x_test_w2v, new_y_test_w2v)


72/72 [==============================] - 1s 9ms/step
Kappa Score: 0.8226728640408434


In [ ]:
aug_x_tr, aug_x_ts, aug_y_tr, aug_y_ts = train_test_split(aug_vec_w2v, aug_y_w2v, test_size=0.2, random_state=41)


In [ ]:
rnn = rnn('LSTM', aug_x_tr.shape)
rnn.fit(aug_x_tr, aug_y_tr, batch_size=64, epochs=100)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (13039, 1, 600)          1442400   
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (13039, 128)             340480    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (13039, 128)              0         
                                                                 
 dense_1 (Dense)             (13039, 1)                129       
                                                                 
Total params: 1,783,009
Trainable params: 1,783,009
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
204/204 [=======================

In [ ]:
rnn_2 = rnn('GRU', aug_x_tr.shape)
rnn_2.fit(aug_x_tr, aug_y_tr, batch_size=64, epochs=100)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_2 (Bidirectio  (13718, 1, 600)          1083600   
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (13718, 128)             255744    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (13718, 128)              0         
                                                                 
 dense_1 (Dense)             (13718, 1)                129       
                                                                 
Total params: 1,339,473
Trainable params: 1,339,473
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
215/215 [=======================

In [ ]:
test_rnn_model(rnn, aug_x_ts, aug_y_ts)


102/102 [==============================] - 2s 11ms/step
Kappa Score: 0.7184912719076009


In [ ]:
test_rnn_model(rnn_2, aug_x_ts, aug_y_ts)


108/108 [==============================] - 2s 9ms/step
Kappa Score: 0.8655998129458755


In [ ]:
test_rnn_model(rnn, X_test_w2v, y_test_w2v)


84/84 [==============================] - 1s 11ms/step
Kappa Score: 0.6790813073851811


In [ ]:
test_rnn_model(rnn_2, X_test_w2v, y_test_w2v)


84/84 [==============================] - 1s 10ms/step
Kappa Score: 0.671551319704633
